<h1 style="text-align: center;">COVID-19 Global Data Analysis & Visualization</h1>

<div style="text-align: center;">
    <p><strong>Presented By:</strong></p>
    <p><strong>Sakib Hosen Himel (947572); Mohammed Mahmudul Hasan (945167); Md Ifte Khairul Islam (948915)</strong></p>
    <p>Date: 31.07.2024</p>
    <p>Data Visualization - Summer 2024</p>
    <p>Instructor: Professor Ulrike Grömping</p>
</div>


<div style="text-align: center;">
    <h2>Table of Contents</h2>
</div>
<ol>
    <li>Introduction</li>
    <li>Datasets Description</li>
    <li>Exploratory Data Visualization and Analysis
        <ul>
            <li>3.1 Introduction to the Pandemic Distribution.</li>
            <li>3.2 Regional Analysis.</li>
            <li>3.3 Tracking the Pandemic Over Time.</li>
            <li>3.4 Deep Dive into Specific Metrics.</li>
            <li>3.5 Comparative Analysis.</li>
            <li>3.6 Monthly and Weekly Trends, Regional.</li>
            <li>3.7 Vaccination and Testing.</li>
            <li>3.8 Hospitalizations and ICU Data.</li>
        </ul>
    </li>
    <li>Conclusion</li>
    <li>References</li>
</ol>


<h1 style="text-align: center;">Introduction</h1>


<p><strong>The COVID-19 pandemic has profoundly impacted the world, reshaping our lives, economies, and healthcare systems. In this report, we are going to use data visualization techniques to provide a comprehensive understanding of the pandemic's trajectory, its regional and global effects, and the critical metrics that have defined our response to this crisis.

This report is structured into nine sections, each addressing key questions through detailed visual analyses. We will explore various dimensions of the pandemic, from the overall global landscape to specific country comparisons, temporal trends, regional disparities, vaccination progress, and healthcare system burdens.</strong></p>

<h1 style="text-align: center;">Data Sources</h1>


<p>For this report, the primary data source was the <strong>*World Health Organization*</strong> (WHO)[1] which provided comprehensive and authoritative data on global COVID-19 cases, deaths. To enrich our analysis and ensure data robustness, we also included supplementary data from <strong>*worldometers.info*</strong> [2], <strong>*Our World in Data*</strong> [3][4] and <strong>*Johns Hopkins University*</strong> [5]. These sources offered additional insights, particularly in areas such as testing rates, vaccination progress, and regional case distributions. By integrating data from these reputable organizations, we aimed to create a more holistic and detailed visualization of the pandemic's impact worldwide.</p>

<h1 style="text-align: center;">Data Description</h1>


<p>
    
- The "worldometer_covid_data.csv" includes detailed country-level data on total cases, deaths, recoveries, active cases, and testing metrics, with additional demographic information. 
    
- "country_daywise.csv" tracks daily case counts, deaths, and recoveries for each country. 
    
- The "vaccination-data.csv" provides vaccination statistics, including total vaccinations, persons vaccinated, and booster doses, along with the types and dates of vaccines used. 
    
- The "WHO-COVID-19-global-data.csv" records daily reported cases and deaths worldwide, including WHO region data. 

- The "owid-covid-data.csv" from Our World in Data offers extensive metrics, including total and new cases, deaths, hospitalizations, testing rates, and various socio-economic indicators. 
    
- Lastly, "covid-hospitalizations.csv" details hospitalization figures by country, date, and specific indicators. These datasets collectively provide a robust framework for analyzing the pandemic's progression and impact globally.</p>

<h1 style="text-align: center;">Import Libraries </h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import requests
from bs4 import BeautifulSoup

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from IPython.display import display, HTML


from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import os
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 70)

/var/folders/sx/scn8rpr941n784f8n2z66qrm0000gn/T/ipykernel_34674/2398226324.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/sx/scn8rpr941n784f8n2z66qrm0000gn/T/ipykernel_34674/2398226324.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<h1 style="text-align: center;">Web Scraping Worldometer Data</h1>


In [ ]:
url = 'https://www.worldometers.info/coronavirus/'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the data
table = soup.find('table', id='main_table_countries_today')

# Extract table headers
headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())

# Extract table rows
rows = []
for tr in table.find_all('tr'):
    cells = tr.find_all('td')
    if len(cells) > 0:
        row = [cell.text.strip() for cell in cells]
        rows.append(row)

df = pd.DataFrame(rows, columns=headers)

print("Data has been scraped and saved to 'worldometer_covid_data.csv'")


# Function to remove commas and convert to numeric
def remove_commas(df):
    for column in df.columns:
        if df[column].dtype == object:
            # Check if the column has any comma-containing strings
            if df[column].str.contains(',').any():
                # Remove commas and convert to numeric
                df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')
                # Check if all values are integers
                if df[column].dropna().apply(float.is_integer).all():
                    df[column] = df[column].astype('Int64')  # Use Int64 to allow for NaNs
    return df

# Apply the function to the DataFrame
df = remove_commas(df)

df = df.fillna(0)

df.to_csv('worldometer_covid_data.csv', index=False)

<h1 style="text-align: center;">Load Datasets</h1>


In [2]:
world_data =pd.read_csv("worldometer_covid_data.csv")
day_wise =pd.read_csv("daywise.csv")
country_daywise =pd.read_csv("country_daywise.csv")
WHO_vac = pd.read_csv('vaccination-data.csv')
WHO_df =pd.read_csv("WHO-COVID-19-global-data.csv")
url='https://covid.ourworldindata.org/data/owid-covid-data.csv'
covid_map =pd.read_csv(url)
covid_data =pd.read_csv("covid-hospitalizations.csv")


<h1 style="text-align: center;">Exploratory Data Visualization and Analysis</h1>


<h1 style="text-align: center;">Introduction to Pandemic</h1>


<h2 style="text-align: center;">Analysis of different countries w.r.t. their Total cases, Deaths, Recovered & Active Cases</h2>

In [3]:
# Define the columns for which you want to create treemaps
columns = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases']
countries = world_data['Country'].unique()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H2("TreeMap Representation of COVID-19 Cases"),
    html.Div([
        html.Div([
            html.Label("Select Metric's"),
            dcc.Dropdown(
                id='column-dropdown',
                options=[{'label': col, 'value': col} for col in columns],
                value=columns[0],  # default value
                style={'width': '100%'}
            )
        ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '10px'}),
        html.Div([
            html.Label("Select Countries"),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in countries],
                value=[],  # default value
                multi=True,
                style={'width': '100%'}
            )
        ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'})
    ]),
    dcc.Graph(id='treemap')
])

# Define callback to update the treemap based on dropdown values
@app.callback(
    dash.dependencies.Output('treemap', 'figure'),
    [dash.dependencies.Input('column-dropdown', 'value'),
     dash.dependencies.Input('country-dropdown', 'value')]
)
def update_treemap(selected_column, selected_countries):
    filtered_data = world_data if not selected_countries else world_data[world_data['Country'].isin(selected_countries)]
    
    if len(filtered_data) == 1:
        color_discrete_sequence = ['#636EFA']  # Set a specific color for single selection
    else:
        color_discrete_sequence = px.colors.qualitative.Dark2  # Use the default color sequence

    fig = px.treemap(
        filtered_data,
        values=selected_column,
        path=['Country'],
        template="plotly_white",
        color_discrete_sequence=color_discrete_sequence,
        title=f"<b>TreeMap representation of Countries w.r.t. their {selected_column}</b>"
    )
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>%{value}<extra></extra>',
        textinfo='label+value'
    )
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25)
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9010)

<h2 style="text-align: center;">Percentage Distribution of Total Confirmed Cases and Total Deaths of Top 20 Countries.</h2>

In [4]:
# Define categories
categories = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases', 'Serious,Critical']

# Use a larger color palette
color_palette = px.colors.qualitative.Alphabet

# Create figure function
def create_figure_percentage(category):
    fig = go.Figure()

    top_20_countries = world_data.sort_values(by=category, ascending=False).iloc[:20]
    total = top_20_countries[category].sum()
    percentages = top_20_countries[category] / total * 100
    text = percentages.round(2).astype(str) + '%'

    fig.add_trace(go.Bar(
        x=top_20_countries['Country'],
        y=top_20_countries[category],
        name=category,
        text=text,
        textposition='outside',
        marker_color=color_palette[:len(top_20_countries)]  # Apply color palette
    ))

    fig.update_layout(
        title=f"COVID-19 Cases: {category}",
        yaxis_title='Number of Cases',
        xaxis_title='Country',
        template="plotly_white",
        hoverlabel=dict(
            bgcolor="skyblue",
            font_size=12,
            font_family="Arial"
        )
    )

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H2("Percentage distribution of countrywise analysis"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in categories],
        value='TotalCases',  # Initial value
        style={'width': '50%'}
    ),
    dcc.Graph(id='coronavirus-cases-graph')
])

# Define callback to update graph based on dropdown selection
@app.callback(
    Output('coronavirus-cases-graph', 'figure'),
    [Input('category-dropdown', 'value')]
)
def update_graph_percentage(selected_category):
    return create_figure_percentage(selected_category)

if __name__ == '__main__':
    app.run_server(debug=True, port=9051)


<h2 style="text-align: center;">Bar Chart Race, Evolution of Total Cases Over Time</h2>

In [5]:
from IPython.display import display, HTML

# HTML code for embedding the Flourish visualization
html_code = """
<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/18852507"><script src="https://public.flourish.studio/resources/embed.js"></script></div>
"""

# Display the HTML in the Jupyter notebook
display(HTML(html_code))

<h2 style="text-align: center;">WHO Region-Wise Percentage Analysis of the Most Affected Continents.</h2>

In [6]:
world_data['Continent'].fillna('Unknown', inplace=True)

# Filter out rows where 'Continent' is 'Unknown'
filtered_data = world_data[world_data['Continent'] != 'Unknown']

cases = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases']

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H2("Coronavirus Cases Analysis by Regions (Parcentagewise)"),
    dcc.Dropdown(
        id='case-dropdown',
        options=[{'label': case, 'value': case} for case in cases],
        value='TotalCases',  # Initial value
        style={'width': '50%'}
    ),
    dcc.Graph(id='pie-chart')
])

# Define callback to update pie chart based on dropdown selection
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('case-dropdown', 'value')]
)
def update_pie_chart(selected_case):
    fig = px.pie(filtered_data, values=selected_case, names='Continent', template="plotly_white", hole=0.3,
                 title="{} Recorded w.r.t. to WHO Region worst affected Continents".format(selected_case))
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=9052)

<h1 style="text-align: center;">Tracking the Pandemic Over Time</h1>


<h2 style="text-align: center;">Evolution of Confirmed, Deaths, Recovered, Active Cases with respect to Time [Jan 2020 - July 2021]</h2>

In [7]:
day_wise['Date'] = pd.to_datetime(day_wise['Date'])
day_wise_filtered = day_wise[day_wise['Date'] <= '2021-07-30']

# Define a custom color palette
custom_colors = ['#393e46', '#ff2e63', '#21bf73', '#fe9801']
fill_colors = ['rgba(255, 154, 0, 0.6)', 'rgba(255, 77, 109, 0.6)', 'rgba(144, 120, 255, 0.6)', 'rgba(54, 38, 167, 0.6)']

# Create the line plot with filled areas
fig = px.line(day_wise_filtered, x='Date', y=['Confirmed', 'Deaths', 'Recovered', 'Active'],
              title='Total Covid Cases reported to WHO (Daily); Jan 2020 - Jul 2021',
              template='plotly_white',
              labels={'variable': 'Cases', 'value': 'Number of Cases'},
              color_discrete_sequence=custom_colors,
              line_group='variable',
              hover_name='variable',
              hover_data={'Date': True, 'variable': False})

# Fill the area under each line with custom fill colors
for i, trace in enumerate(fig.data):
    trace.update(fill='tonexty', fillcolor=fill_colors[i])

# Customize layout for horizontal grid lines only and set figure size
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of Cases',
    xaxis=dict(showgrid=False),  # Disable vertical grid lines
    yaxis=dict(showgrid=True),   # Enable horizontal grid lines
    width=950,  # Set the width of the figure
    height=500   # Set the height of the figure
)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='covid-line-plot', figure=fig)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=9021)

<h1 style="text-align: center;">Deep Dive into Specific Metrics</h1>


<h2 style="text-align: center;">Analysis of Population-to-Tests Ratio in the twenty countries with the highest COVID-19 case counts.</h2>


In [8]:
# Calculate population to tests done ratio
pop_test_ratio = world_data['Population'] / world_data['TotalTests']

# Add the calculated ratio to the DataFrame for sorting
world_data['pop_test_ratio'] = pop_test_ratio

# Sort the DataFrame by the pop_test_ratio in descending order
world_data_sorted = world_data.iloc[0:20].sort_values(by='pop_test_ratio', ascending=False)

# Format the pop_test_ratio for display
world_data_sorted['pop_test_ratio_formatted'] = (world_data_sorted['pop_test_ratio']).round(2).astype(str) + '%'

# Create the bar chart
fig = px.bar(world_data_sorted, color='pop_test_ratio', y='pop_test_ratio', x='Country',
             template="plotly_white", title="<b>20 highest COVID-19 countries population to tests done ratio</b>")

# Add the text on top of each bar
fig.update_traces(text=world_data_sorted['pop_test_ratio_formatted'], textposition='outside')

# Update layout
fig.update_layout(yaxis_title='Ratios(%)')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='covid-bar-plot', figure=fig)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=9022)

<h2 style="text-align: center;">WHO Continents, COVID-19 Ratios of Confirmed, Deaths, Recovered, Serious Cases.</h2>


In [9]:
app = dash.Dash(__name__)

# Define dropdown options
ratio_options = [
    {'label': 'Death to confirmed ratio', 'value': 'death_to_confirmed'},
    {'label': 'Death to recovered ratio', 'value': 'death_to_recovered'},
    {'label': 'Tests to confirmed ratio', 'value': 'tests_to_confirmed'},
    {'label': 'Serious to death ratio', 'value': 'serious_to_death'}
]

country_options = [{'label': country, 'value': country} for country in world_data['Country'].unique()]

# Layout of the Dash app
app.layout = html.Div([
    html.H2("COVID-19 Ratios Dashboard"),
    
    html.Div([
        html.Div([
            html.Label("Select Ratio:"),
            dcc.Dropdown(
                id='ratio-dropdown',
                options=ratio_options,
                value='death_to_confirmed',  # Default value
                clearable=False
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Countries:"),
            dcc.Dropdown(
                id='country-dropdown',
                options=country_options,
                value=[],  # Default value (initially empty list)
                multi=True
            )
        ], style={'width': '48%', 'display': 'inline-block', 'marginLeft': '4%'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    
    # Checkbox for logarithmic scale
    dcc.Checklist(
        id='log-scale-checkbox',
        options=[{'label': 'Logarithmic scale', 'value': 'log'}],
        value=[]
    ),
    
    # Placeholder for the graph
    dcc.Graph(id='ratio-graph')
])

# Callback to update graph based on dropdown value and checkbox
@app.callback(
    Output('ratio-graph', 'figure'),
    [Input('ratio-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('log-scale-checkbox', 'value')]
)
def update_graph(selected_ratio, selected_countries, log_scale):
    if not selected_countries:
        filtered_data = world_data
    else:
        filtered_data = world_data[world_data['Country'].isin(selected_countries)]
    
    if selected_ratio == 'death_to_confirmed':
        y_data = filtered_data['TotalDeaths'] / filtered_data['TotalCases']
        title = "Death to confirmed ratio of selected countries"
    elif selected_ratio == 'death_to_recovered':
        y_data = filtered_data['TotalDeaths'] / filtered_data['TotalRecovered']
        title = "Death to recovered ratio of selected countries"
    elif selected_ratio == 'tests_to_confirmed':
        y_data = filtered_data['TotalTests'] / filtered_data['TotalCases']
        title = "Tests to confirmed ratio of selected countries"
    elif selected_ratio == 'serious_to_death':
        y_data = filtered_data['Serious,Critical'] / filtered_data['TotalDeaths']
        title = "Serious to Death ratio of selected countries"

    fig = px.scatter(filtered_data, x='Country', y=y_data, color='Country', title=title, template="plotly_white")
    
    # Update layout
    fig.update_layout(
        title={'text': title, 'xanchor': 'left'},
        yaxis_title='Ratios',
        xaxis_title='Countries',
        xaxis=dict(showgrid=False), 
        yaxis=dict(showgrid=True, type='log' if 'log' in log_scale else 'linear', tickformat=".2f")
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9013)

<h1 style="text-align: center;">Comparative Analysis</h1>


<h2 style="text-align: center;"> Entire statistics confirmed, active, deaths cases of a particular country (Germany).</h2>


In [10]:
country_daywise['Date'] = pd.to_datetime(country_daywise['Date'])

# Define the function to create the visualizations
def country_visualization_1(country_daywise, country):
    data = country_daywise[country_daywise['Country'] == country]
    df = data.loc[:, ['Date', 'Confirmed', 'Deaths', 'Active']]
    
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Confirmed", "Active", "Deaths"))
    
    fig.add_trace(
        go.Scatter(
            name="Confirmed",
            x=df['Date'],
            y=df['Confirmed'],
            hovertemplate='Date: %{x}<br>Confirmed: %{y}<extra></extra>'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            name="Active",
            x=df['Date'],
            y=df['Active'],
            hovertemplate='Date: %{x}<br>Active: %{y}<extra></extra>'
        ),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(
            name="Deaths",
            x=df['Date'],
            y=df['Deaths'],
            hovertemplate='Date: %{x}<br>Deaths: %{y}<extra></extra>'
        ),
        row=1, col=3
    )
    
    fig.update_layout(
        height=500,
        width=950,
        title_text="Date Vs Recorded Cases of {}".format(country),
        template="plotly_white",
        xaxis=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis2=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis2=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis3=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis3=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
    )

    # Set range to start from 0 for y-axis
    fig.update_yaxes(rangemode='tozero')

    return fig

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H2("Entire statistics confirmed, active, deaths cases of a particular country."),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in country_daywise['Country'].unique()],
        value='Germany'  # Set a default value
    ),
    dcc.Graph(id='country-graph')
])

# Define callback to update the graph based on selected country
@app.callback(
    Output('country-graph', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_figure_germany(selected_country):
    return country_visualization_1(country_daywise, selected_country)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9042)

<h2 style="text-align: center;">Entire statistics confirmed, active , deaths cases comparisam of two unique countries.</h2>

In [11]:
country_daywise['Date'] = pd.to_datetime(country_daywise['Date'])

# Define the function to create the visualizations
def country_visualization_2(country_daywise, country1, country2):
    # Filter data for the two countries
    data_country1 = country_daywise[country_daywise['Country'] == country1]
    data_country2 = country_daywise[country_daywise['Country'] == country2]

    # Define colors for the countries
    color_country1 = 'blue'
    color_country2 = 'red'

    # Create subplots with shared y-axis
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Confirmed", "Active", "Deaths"))

    # Add traces for each statistic with improved hover info
    fig.add_trace(go.Scatter(
        name=country1 + " Confirmed",
        x=data_country1['Date'],
        y=data_country1['Confirmed'],
        line=dict(color=color_country1),
        hovertemplate='Country: ' + country1 + '<br>Date: %{x}<br>Confirmed: %{y}<extra></extra>'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        name=country2 + " Confirmed",
        x=data_country2['Date'],
        y=data_country2['Confirmed'],
        line=dict(color=color_country2),
        hovertemplate='Country: ' + country2 + '<br>Date: %{x}<br>Confirmed: %{y}<extra></extra>'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        name=country1 + " Active",
        x=data_country1['Date'],
        y=data_country1['Active'],
        line=dict(color=color_country1),
        hovertemplate='Country: ' + country1 + '<br>Date: %{x}<br>Active: %{y}<extra></extra>'
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        name=country2 + " Active",
        x=data_country2['Date'],
        y=data_country2['Active'],
        line=dict(color=color_country2),
        hovertemplate='Country: ' + country2 + '<br>Date: %{x}<br>Active: %{y}<extra></extra>'
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        name=country1 + " Deaths",
        x=data_country1['Date'],
        y=data_country1['Deaths'],
        line=dict(color=color_country1),
        hovertemplate='Country: ' + country1 + '<br>Date: %{x}<br>Deaths: %{y}<extra></extra>'
    ), row=1, col=3)
    fig.add_trace(go.Scatter(
        name=country2 + " Deaths",
        x=data_country2['Date'],
        y=data_country2['Deaths'],
        line=dict(color=color_country2),
        hovertemplate='Country: ' + country2 + '<br>Date: %{x}<br>Deaths: %{y}<extra></extra>'
    ), row=1, col=3)

    # Update layout
    fig.update_layout(
        height=480,
        width=950,
        title_text=f"Date Vs Recorded Cases of {country1} vs {country2}",
        template="plotly_white",
        xaxis=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis2=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis2=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis3=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis3=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
    )

    # Set range to start from 0 for y-axis
    fig.update_yaxes(rangemode='tozero')

    return fig

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H2("Entire statistics confirmed, active, deaths cases of two unique countries."),
    html.Label("Select First Country:"),
    dcc.Dropdown(
        id='country1-dropdown',
        options=[{'label': country, 'value': country} for country in country_daywise['Country'].unique()],
        value='Italy'  # Set a default value
    ),
    html.Label("Select Second Country:"),
    dcc.Dropdown(
        id='country2-dropdown',
        options=[{'label': country, 'value': country} for country in country_daywise['Country'].unique()],
        value='Germany'  # Set a default value
    ),
    dcc.Graph(id='country-comparison-graph')
])

# Define callback to update the graph based on selected countries
@app.callback(
    Output('country-comparison-graph', 'figure'),
    [Input('country1-dropdown', 'value'),
     Input('country2-dropdown', 'value')]
)
def update_figure_comparisam(selected_country1, selected_country2):
    return country_visualization_2(country_daywise, selected_country1, selected_country2)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9053)

<h2 style="text-align: center;">Entire statistics confirmed, active, recovered , deaths cases of a particular country Vs World's</h2>

In [12]:
country_daywise['Date'] = pd.to_datetime(country_daywise['Date'])

# Define the function to create the visualizations
def country_visualization_3(country_daywise, country):
    # Filter data for the selected country
    data_country = country_daywise[country_daywise['Country'] == country]
    
    # Filter data for all other countries
    data_others = country_daywise[country_daywise['Country'] != country]
    
    # Aggregate data for all other countries
    data_aggregated = data_others.groupby('Date').agg({
        'Confirmed': 'sum',
        'Deaths': 'sum',
        'Active': 'sum'
    }).reset_index()
    
    # Define colors for the country and the world
    color_country = 'blue'
    color_world = 'green'
    
    # Create subplots with shared y-axis
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Confirmed", "Active", "Deaths"))

    # Add traces for the selected country with improved hover info
    fig.add_trace(go.Scatter(
        name=f"{country} Confirmed",
        x=data_country['Date'],
        y=data_country['Confirmed'],
        line=dict(color=color_country),
        hovertemplate='Country: ' + country + '<br>Date: %{x}<br>Confirmed: %{y}<extra></extra>'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        name=f"{country} Active",
        x=data_country['Date'],
        y=data_country['Active'],
        line=dict(color=color_country),
        hovertemplate='Country: ' + country + '<br>Date: %{x}<br>Active: %{y}<extra></extra>'
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        name=f"{country} Deaths",
        x=data_country['Date'],
        y=data_country['Deaths'],
        line=dict(color=color_country),
        hovertemplate='Country: ' + country + '<br>Date: %{x}<br>Deaths: %{y}<extra></extra>'
    ), row=1, col=3)

    # Add traces for aggregated data of all other countries with improved hover info
    fig.add_trace(go.Scatter(
        name="World Confirmed",
        x=data_aggregated['Date'],
        y=data_aggregated['Confirmed'],
        line=dict(color=color_world),
        hovertemplate='World<br>Date: %{x}<br>Confirmed: %{y}<extra></extra>'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        name="World Active",
        x=data_aggregated['Date'],
        y=data_aggregated['Active'],
        line=dict(color=color_world),
        hovertemplate='World<br>Date: %{x}<br>Active: %{y}<extra></extra>'
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        name="World Deaths",
        x=data_aggregated['Date'],
        y=data_aggregated['Deaths'],
        line=dict(color=color_world),
        hovertemplate='World<br>Date: %{x}<br>Deaths: %{y}<extra></extra>'
    ), row=1, col=3)

    # Update layout
    fig.update_layout(
        height=500,
        width=950,
        title_text=f"Date Vs Recorded Cases of {country} vs World",
        template="plotly_white",
        xaxis=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis2=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis2=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
        xaxis3=dict(linecolor='black', showgrid=True, zerolinecolor='black'),
        yaxis3=dict(linecolor='black', showgrid=True, zeroline=True, zerolinecolor='black'),
    )

    # Set range to start from 0 for y-axis
    fig.update_yaxes(rangemode='tozero')

    return fig

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H2("Entire statistics confirmed, active, recovered , deaths cases; single country vs world"),
    html.Label("Select Country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in country_daywise['Country'].unique()],
        value='US'  # Set a default value
    ),
    dcc.Graph(id='country-world-comparison-graph')
])

# Define callback to update the graph based on selected country
@app.callback(
    Output('country-world-comparison-graph', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_figure_world(selected_country):
    return country_visualization_3(country_daywise, selected_country)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True ,port=9024)

<h1 style="text-align: center;">Monthly and Weekly Trends with Regions</h1>


<h2 style="text-align: center;">Monthly total confirmed and deaths COVID-19 cases per million people with respect to Continents.</h2>

In [13]:
# Convert 'date' to datetime and prepare a display label
covid_map['date'] = pd.to_datetime(covid_map['date'])
covid_map['date_label'] = covid_map['date'].dt.strftime('%b %Y')  # Format as 'Abbreviated Month YYYY'

# Define a custom color scale
custom_color_scale = [
    [0.0, "#f7fcf0"],   # light yellow
    [0.1, "#e0f3db"],   # slightly darker yellow
    [0.2, "#ccebc5"],   # light green
    [0.3, "#a8ddb5"],   # green
    [0.4, "#7bccc4"],   # light blue-green
    [0.5, "#4eb3d3"],   # blue-green
    [0.6, "#2b8cbe"],   # darker blue-green
    [0.7, "#0868ac"],   # blue
    [0.8, "#084081"],   # dark blue
    [1.0, "#062d64"]    # very dark blue
]

# Center and zoom settings for each continent
continent_settings = {
    'Africa': {'center': {'lat': 0, 'lon': 20}, 'zoom': 2},
    'Asia': {'center': {'lat': 34, 'lon': 100}, 'zoom': 1.5},
    'Europe': {'center': {'lat': 70, 'lon': 25}, 'zoom': 1.5},
    'North America': {'center': {'lat': 40, 'lon': -100}, 'zoom': 2},
    'South America': {'center': {'lat': -15, 'lon': -60}, 'zoom': 2},
    'Oceania': {'center': {'lat': -25, 'lon': 140}, 'zoom': 2}
}

# Create the choropleth map function to reuse for cases and deaths
def create_choropleth(metric, continent=None):
    df = covid_map if continent is None else covid_map[covid_map['continent'] == continent]
    fig = px.choropleth(
        df,
        locations="location",
        color=metric,
        locationmode='country names',
        hover_name="location",
        hover_data={'date_label': True, 'location': False, metric: ':.2f'},
        color_continuous_scale=custom_color_scale,
        animation_frame="date_label",
        projection="natural earth",
        title=f'Evolution of {metric.replace("_per_million", "").replace("_", " ")} in each country',
        template='plotly_white'
    )
    if continent:
        fig.update_geos(
            center=continent_settings[continent]['center'],
            projection_scale=continent_settings[continent]['zoom']
        )
    fig.update_traces(
        hovertemplate='<b>Country: %{hovertext}</b><br>Date: %{customdata[0]}<br>' + metric.replace('_', ' ').title() + ': %{z:.2f}'
    )
    fig.update_layout(
        width=1000,  # Adjust width
        height=600   # Adjust height
    )
    return fig

app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='metric-select',
        options=[
            {'label': 'Total Cases per Million', 'value': 'total_cases_per_million'},
            {'label': 'Total Deaths per Million', 'value': 'total_deaths_per_million'}
        ],
        value='total_cases_per_million'  # Default value
    ),
    dcc.Dropdown(
        id='continent-select',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Africa', 'value': 'Africa'},
            {'label': 'Asia', 'value': 'Asia'},
            {'label': 'Europe', 'value': 'Europe'},
            {'label': 'North America', 'value': 'North America'},
            {'label': 'South America', 'value': 'South America'},
            {'label': 'Oceania', 'value': 'Oceania'}
        ],
        value='All'  # Default value
    ),
    dcc.Graph(id='covid-map')
])

# Callback to update the map based on the dropdown selections
@app.callback(
    Output('covid-map', 'figure'),
    [Input('metric-select', 'value'), Input('continent-select', 'value')]
)
def update_map(selected_metric, selected_continent):
    if selected_continent == 'All':
        selected_continent = None
    return create_choropleth(selected_metric, selected_continent)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9014)

In [14]:
covid_map['date'] = pd.to_datetime(covid_map['date'])
covid_map['date_label'] = covid_map['date'].dt.strftime('%b %Y')  # Format as 'Abbreviated Month YYYY'

# Define a custom color scale
custom_color_scale = [
    "#f7fcf0",   # light yellow
    "#e0f3db",   # slightly darker yellow
    "#ccebc5",   # light green
    "#a8ddb5",   # green
    "#7bccc4",   # light blue-green
    "#4eb3d3",   # blue-green
    "#2b8cbe",   # darker blue-green
    "#0868ac",   # blue
    "#084081",   # dark blue
    "#062d64"    # very dark blue
]

# Create the bar chart function for the top 10 countries
def create_bar_chart_covid(metric):
    latest_date = covid_map['date'].max()
    df_latest = covid_map[covid_map['date'] == latest_date].sort_values(by=metric, ascending=False).head(10)
    fig = px.bar(
        df_latest,
        x=metric,
        y='location',
        orientation='h',
        title=f'Top 10 Countries by {metric.replace("_per_million", "").replace("_", " ").title()}',
        text=metric,
        color=metric,
        color_continuous_scale=custom_color_scale,
        template='plotly_white'
    )
    fig.update_traces(
        texttemplate='%{text:.2f}',
        textposition='inside'
    )
    fig.update_layout(
        xaxis_title=metric.replace('_', ' ').title(),
        yaxis_title='Country',
        width=900,  # Adjust width
        height=500   # Adjust height
    )
    return fig

app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='metric-select',
        options=[
            {'label': 'Total Cases per Million', 'value': 'total_cases_per_million'},
            {'label': 'Total Deaths per Million', 'value': 'total_deaths_per_million'}
        ],
        value='total_cases_per_million'  # Default value
    ),
    dcc.Graph(id='bar-chart')
])

# Callback to update the bar chart based on the selected metric
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('metric-select', 'value')]
)
def update_bar_chart_covid(selected_metric):
    return create_bar_chart_covid(selected_metric)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9084)


<h2 style="text-align: center;">Trends in COVID-19 New Cases Worldwide reported to WHO (weekly).
</h2>

In [28]:
WHO_df['Date_reported'] = pd.to_datetime(WHO_df['Date_reported'])
WHO_df['New_cases'] = pd.to_numeric(WHO_df['New_cases'], errors='coerce')

# Aggregate data by week for recent cases (July 2023 - present)
recent_data = WHO_df[WHO_df['Date_reported'] >= '2023-07-01']
recent_weekly_cases = recent_data.groupby(pd.Grouper(key='Date_reported', freq='W-SUN'))['New_cases'].sum().reset_index()

# Aggregate data by week for total cases (January 2020 - present)
total_weekly_cases = WHO_df.groupby(pd.Grouper(key='Date_reported', freq='W-SUN'))['New_cases'].sum().reset_index()

# Find the highest peak for recent cases
max_recent_index = recent_weekly_cases['New_cases'].idxmax()
max_recent_date = recent_weekly_cases.at[max_recent_index, 'Date_reported']
max_recent_value = recent_weekly_cases.at[max_recent_index, 'New_cases']

# Find the highest peak for total cases
max_total_index = total_weekly_cases['New_cases'].idxmax()
max_total_date = total_weekly_cases.at[max_total_index, 'Date_reported']
max_total_value = total_weekly_cases.at[max_total_index, 'New_cases']

# Plot recent COVID-19 cases (weekly)
fig1 = px.bar(recent_weekly_cases, x='Date_reported', y='New_cases', 
              title='Recent COVID-19 cases reported to WHO (weekly)',
              labels={'New_cases': 'New Cases'},
              template='plotly_white')

fig1.update_traces(marker_color=['black' if i == max_recent_index else '#51a9ed' for i in range(len(recent_weekly_cases))],
                   marker_line_color='rgb(8,48,107)', marker_line_width=1, opacity=0.9)
fig1.update_yaxes(tickformat=",", tickvals=[100000, 200000, 300000, 400000, 500000, 600000, 700000], 
                  ticktext=["100k", "200k", "300k", "400k", "500k", "600k", "700k"],
                  showgrid=True, gridwidth=0.1, gridcolor='lightgray', griddash='dot')
fig1.update_layout(xaxis_title=None)

# Add subtitle to the first figure
fig1.add_annotation(
    x=0.5,
    y=1.05,
    xref='paper',
    yref='paper',
    showarrow=False,
    text="World, July 2023 - present",
    font=dict(size=12)
)

# Plot total COVID-19 cases (weekly)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=total_weekly_cases['Date_reported'], y=total_weekly_cases['New_cases'], 
                          fill='tozeroy', fillcolor='rgba(135,206,250,0.5)', 
                          line=dict(color='black'), 
                          mode='lines'))

fig2.update_layout(title='Total COVID-19 cases reported to WHO (weekly)', 
                   yaxis_title='New Cases(M)',
                   template='plotly_white',
                   xaxis_title=None)

fig2.update_yaxes(tickformat=",", tickvals=[10000000, 20000000, 30000000, 40000000], 
                  ticktext=["10m", "20m", "30m", "40m"],
                  showgrid=True, gridwidth=0.1, gridcolor='lightgray', griddash='dot')

# Add subtitle to the second figure
fig2.add_annotation(
    x=0.5,
    y=1.05,
    xref='paper',
    yref='paper',
    showarrow=False,
    text="World, January 2020 - present",
    font=dict(size=12)
)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='recent-covid-cases', figure=fig2),
    dcc.Graph(id='total-covid-cases', figure=fig1)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9025)


<h2 style="text-align: center;">Number of weekly COVID-19 New Cases reported to WHO (Regional).
</h2>

In [25]:
WHO_df['Date_reported'] = pd.to_datetime(WHO_df['Date_reported'])
WHO_df = WHO_df[WHO_df['New_cases'].notna()]
WHO_df = WHO_df.dropna(subset=['WHO_region'])
WHO_df = WHO_df[~WHO_df['WHO_region'].isin(['OTHER'])]

# Map region codes to full names
region_names = {
    'AFRO': 'Africa',
    'AMRO': 'Americas',
    'EMRO': 'Eastern Mediterranean',
    'EURO': 'Europe',
    'SEARO': 'South-East Asia',
    'WPRO': 'Western Pacific'
}
WHO_df['Region_Name'] = WHO_df['WHO_region'].map(region_names)

# Group data by region and date, summing new cases for each week
df_weekly = WHO_df.groupby(['Region_Name', pd.Grouper(key='Date_reported', freq='W-MON')])['New_cases'].sum().reset_index()

# Function to format y-axis values in thousands (K) and millions (M)
def format_y_axis(value):
    if value >= 1_000_000:
        return f'{value // 1_000_000}M'
    elif value >= 1000:
        return f'{value // 1_000}K'
    else:
        return f'{value:.0f}'

# Function to round up to the nearest standard threshold for y-axis ticks
def round_up_to_nearest_standard(value):
    thresholds = [1_000, 2_000, 5_000, 10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000, 2_000_000, 5_000_000, 10_000_000, 20_000_000, 50_000_000]
    for threshold in thresholds:
        if value <= threshold:
            return threshold
    return thresholds[-1]

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("COVID-19 New Cases by WHO Region"),
    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': region, 'value': region} for region in df_weekly['Region_Name'].unique()],
        value=df_weekly['Region_Name'].unique()[0],
        clearable=False
    ),
    dcc.Graph(id='covid-cases-graph')
])

@app.callback(
    Output('covid-cases-graph', 'figure'),
    Input('region-dropdown', 'value')
)
def plot_covid_cases_by_region(region):
    region_colors = {
        'Africa': 'orange',
        'Americas': 'purple',
        'Eastern Mediterranean': 'brown',
        'Europe': 'blue',
        'South-East Asia': 'green',
        'Western Pacific': 'gold'
    }
    
    df_region = df_weekly[df_weekly['Region_Name'] == region]
    
    fig = px.line(df_region, x='Date_reported', y='New_cases', color='Region_Name',
                  color_discrete_map=region_colors, title=f'Number of Weekly COVID-19 Cases in {region} Region Reported to WHO',
                  labels={'New_cases': 'Cases', 'Date_reported': 'Date', 'Region_Name': 'Region'},
                  height=500, width=1000, template='plotly_white')
    
    y_max = df_region['New_cases'].max()
    y_max_rounded = round_up_to_nearest_standard(y_max)
    tickvals = np.arange(0, y_max_rounded + 1, y_max_rounded // 5)
    ticktext = [format_y_axis(val) for val in tickvals]
    
    fig.update_layout(
        yaxis=dict(tickmode='array', tickvals=tickvals, ticktext=ticktext, showgrid=True, gridcolor='lightgray', gridwidth=0.1, griddash='dot'),
        xaxis=dict(showgrid=False),
        yaxis_title='Cases', xaxis_title='', title_font_size=16,
        showlegend=False
    )
    
    fig.update_xaxes(
        dtick="M3",
        tickformat="%b\n%Y",
        ticklabelmode="period"
    )
    
    fig.update_traces(line=dict(width=1), fill='tozeroy', hovertemplate='Date: %{x|%b %d, %Y}<br>Cases: %{y}')
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True,port=9081)


<h2 style="text-align: center;">Trends in COVID-19 New Deaths Worldwide reported to WHO (weekly).
</h2>

In [27]:
WHO_df['Date_reported'] = pd.to_datetime(WHO_df['Date_reported'])
WHO_df['New_deaths'] = pd.to_numeric(WHO_df['New_deaths'], errors='coerce')

# Aggregate data by week for recent deaths (July 2023 - present)
recent_data = WHO_df[WHO_df['Date_reported'] >= '2023-07-01']
recent_weekly_deaths = recent_data.groupby(pd.Grouper(key='Date_reported', freq='W-SUN'))['New_deaths'].sum().reset_index()

# Aggregate data by week for total deaths (January 2020 - present)
total_weekly_deaths = WHO_df.groupby(pd.Grouper(key='Date_reported', freq='W-SUN'))['New_deaths'].sum().reset_index()

# Find the highest peak for recent deaths
max_recent_index = recent_weekly_deaths['New_deaths'].idxmax()
max_recent_date = recent_weekly_deaths.at[max_recent_index, 'Date_reported']
max_recent_value = recent_weekly_deaths.at[max_recent_index, 'New_deaths']

# Find the highest peak for total deaths
max_total_index = total_weekly_deaths['New_deaths'].idxmax()
max_total_date = total_weekly_deaths.at[max_total_index, 'Date_reported']
max_total_value = total_weekly_deaths.at[max_total_index, 'New_deaths']

# Plot recent COVID-19 deaths (weekly)
fig1 = px.bar(recent_weekly_deaths, x='Date_reported', y='New_deaths', 
              title='Recent COVID-19 deaths reported to WHO (weekly)',
              labels={'New_deaths': 'Deaths'},
              template='plotly_white')

fig1.update_traces(marker_color=['black' if i == max_recent_index else '#51a9ed' for i in range(len(recent_weekly_deaths))],
                   marker_line_color='rgb(8,48,107)', marker_line_width=1, opacity=0.9)
fig1.update_yaxes(tickformat=",", tickvals=[1000, 2000, 3000, 4000, 5000, 6000, 7000], 
                  ticktext=["1k", "2k", "3k", "4k", "5k", "6k", "7k"],
                  showgrid=True, gridwidth=0.1, gridcolor='lightgray', griddash='dot')
fig1.update_layout(xaxis_title=None)

# Add subtitle to the first figure
fig1.add_annotation(
    x=0.5,
    y=1.05,
    xref='paper',
    yref='paper',
    showarrow=False,
    text="World, July 2023 - present",
    font=dict(size=12)
)

# Plot total COVID-19 deaths (weekly)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=total_weekly_deaths['Date_reported'], y=total_weekly_deaths['New_deaths'], 
                          fill='tozeroy', fillcolor='rgba(135,206,250,0.5)', 
                          line=dict(color='black'), 
                          mode='lines'))

fig2.update_layout(title='Total COVID-19 deaths reported to WHO (weekly)', 
                   yaxis_title='New Deaths(M)',
                   template='plotly_white',
                   xaxis_title=None)

fig2.update_yaxes(tickformat=",", tickvals=[10000, 20000, 30000, 40000, 60000, 80000, 100000], 
                  ticktext=["10k", "20k", "30k", "40k", "60k", "80k", "100k"],
                  showgrid=True, gridwidth=0.1, gridcolor='lightgray', griddash='dot')

# Add subtitle to the second figure
fig2.add_annotation(
    x=0.5,
    y=1.05,
    xref='paper',
    yref='paper',
    showarrow=False,
    text="World, January 2020 - present",
    font=dict(size=12)
)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='recent-covid-deaths', figure=fig2),
    dcc.Graph(id='total-covid-deaths', figure=fig1)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9027)


<h2 style="text-align: center;">Number of weekly COVID-19 New Deaths reported to WHO (Regional).
</h2>

In [26]:
WHO_df['Date_reported'] = pd.to_datetime(WHO_df['Date_reported'])
WHO_df = WHO_df[WHO_df['New_deaths'].notna()]
WHO_df = WHO_df.dropna(subset=['WHO_region'])
WHO_df = WHO_df[~WHO_df['WHO_region'].isin(['OTHER'])]

# Map region codes to full names
region_names = {
    'AFRO': 'Africa',
    'AMRO': 'Americas',
    'EMRO': 'Eastern Mediterranean',
    'EURO': 'Europe',
    'SEARO': 'South-East Asia',
    'WPRO': 'Western Pacific'
}
WHO_df['Region_Name'] = WHO_df['WHO_region'].map(region_names)

# Group data by region and date, summing new cases for each week
df_weekly_deaths = WHO_df.groupby(['Region_Name', pd.Grouper(key='Date_reported', freq='W-MON')])['New_deaths'].sum().reset_index()

# Function to format y-axis values in thousands (K) and millions (M)
def format_y_axis_deaths(value):
    if value >= 1_000_000:
        return f'{value // 1_000_000}M'
    elif value >= 1000:
        return f'{value // 1_000}K'
    else:
        return f'{value:.0f}'

# Function to round up to the nearest standard threshold for y-axis ticks
def round_up_to_nearest_standard_deaths(value):
    thresholds = [1_000, 2_000, 5_000, 10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000, 2_000_000, 5_000_000, 10_000_000, 20_000_000, 50_000_000]
    for threshold in thresholds:
        if value <= threshold:
            return threshold
    return thresholds[-1]

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("COVID-19 New Deaths by WHO Region"),
    dcc.Dropdown(
        id='region-dropdown-deaths',
        options=[{'label': region, 'value': region} for region in df_weekly_deaths['Region_Name'].unique()],
        value=df_weekly_deaths['Region_Name'].unique()[0],
        clearable=False
    ),
    dcc.Graph(id='covid-deaths-graph')
])

@app.callback(
    Output('covid-deaths-graph', 'figure'),
    Input('region-dropdown-deaths', 'value')
)
def plot_covid_deaths_by_region(region):
    region_colors = {
        'Africa': 'orange',
        'Americas': 'purple',
        'Eastern Mediterranean': 'brown',
        'Europe': 'blue',
        'South-East Asia': 'green',
        'Western Pacific': 'gold'
    }
    
    df_region = df_weekly_deaths[df_weekly_deaths['Region_Name'] == region]
    
    fig = px.line(df_region, x='Date_reported', y='New_deaths', color='Region_Name',
                  color_discrete_map=region_colors, title=f'Number of Weekly COVID-19 Deaths in {region} Region Reported to WHO',
                  labels={'New_deaths': 'Deaths', 'Date_reported': 'Date', 'Region_Name': 'Region'},
                  height=500, width=1000, template='plotly_white')
    
    y_max = df_region['New_deaths'].max()
    y_max_rounded = round_up_to_nearest_standard_deaths(y_max)
    tickvals = np.arange(0, y_max_rounded + 1, y_max_rounded // 5)
    ticktext = [format_y_axis_deaths(val) for val in tickvals]
    
    fig.update_layout(
        yaxis=dict(tickmode='array', tickvals=tickvals, ticktext=ticktext, showgrid=True, gridcolor='lightgray', gridwidth=0.1, griddash='dot'),
        xaxis=dict(showgrid=False),
        yaxis_title='Deaths', xaxis_title='', title_font_size=16,
        showlegend=False
    )
    
    fig.update_xaxes(
        dtick="M3",
        tickformat="%b\n%Y",
        ticklabelmode="period"
    )
    
    fig.update_traces(line=dict(width=1), fill='tozeroy', hovertemplate='Date: %{x|%b %d, %Y}<br>Deaths: %{y}')
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True,port=9082)

<h1 style="text-align: center;">Vaccination, Testing and Stringency</h1>


<h2 style="text-align: center;">Percentage of total population vaccinated with at least one dose, at least one booster dose, and with complete primary series of a COVID-19 vaccine.</h2>

In [19]:
# Filter out rows with NaN values in the relevant columns
WHO_vac_booster = WHO_vac[WHO_vac['PERSONS_BOOSTER_ADD_DOSE_PER100'].notna()]
WHO_vac_first_dose = WHO_vac[WHO_vac['PERSONS_VACCINATED_1PLUS_DOSE_PER100'].notna()]
WHO_vac_complete_primary = WHO_vac[WHO_vac['PERSONS_LAST_DOSE_PER100'].notna()]

# Prepare the data for plotting the maps
data_booster = WHO_vac_booster[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_BOOSTER_ADD_DOSE_PER100']]
data_first_dose = WHO_vac_first_dose[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100']]
data_complete_primary = WHO_vac_complete_primary[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_LAST_DOSE_PER100']]

# Create the choropleth maps
def create_figure_vaccine(data, color, title):
    return px.choropleth(
        data,
        locations='ISO3',
        color=color,
        hover_name='COUNTRY',
        hover_data={'ISO3': False, color: False},
        color_continuous_scale='Blues',
        title=title,
        labels={color: ''},
        projection='natural earth'
    )

fig_booster = create_figure_vaccine(data_booster, 'PERSONS_BOOSTER_ADD_DOSE_PER100', 'Percentage of total population vaccinated with at least one booster dose<br>World, June 2024')
fig_first_dose = create_figure_vaccine(data_first_dose, 'PERSONS_VACCINATED_1PLUS_DOSE_PER100', 'Percentage of total population vaccinated with at least one dose of a COVID-19 vaccine<br>World, June 2024')
fig_complete_primary = create_figure_vaccine(data_complete_primary, 'PERSONS_LAST_DOSE_PER100', 'Percentage of total population vaccinated with a complete primary series<br>World, June 2024')

# Update layout for all maps
for fig in [fig_booster, fig_first_dose, fig_complete_primary]:
    fig.update_layout(
        width=950,
        height=580,
        coloraxis_colorbar=dict(
            title="% of total population",
            orientation="h",
            yanchor="bottom",
            y=-0.25,
            xanchor="center",
            x=0.4,
            ticks="outside",
            len=0.6
        )
    )
    fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>%{z:.1f}%')

# Center and zoom settings for each continent
continent_settings = {
    'Africa': {'center': {'lat': 0, 'lon': 20}, 'zoom': 2},
    'Americas': {'center': {'lat': 10, 'lon': -55}, 'zoom': 1.5},
    'Eastern Mediterranean': {'center': {'lat': 25, 'lon': 45}, 'zoom': 2},
    'Europe': {'center': {'lat': 54, 'lon': 25}, 'zoom': 2},
    'South-East Asia': {'center': {'lat': 10, 'lon': 90}, 'zoom': 2},
    'Western Pacific': {'center': {'lat': 0, 'lon': 140}, 'zoom': 2}
}

# Mapping WHO_REGION to region names
region_names = {
    'AFRO': 'Africa',
    'AMRO': 'Americas',
    'EMRO': 'Eastern Mediterranean',
    'EURO': 'Europe',
    'SEARO': 'South-East Asia',
    'WPRO': 'Western Pacific'
}

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='map-dropdown',
        options=[
            {'label': 'At least one dose', 'value': 'first_dose'},
            {'label': 'At least one booster', 'value': 'booster'},
            {'label': 'Complete primary series', 'value': 'complete_primary'}
        ],
        value='first_dose',
        clearable=False,
        style={'width': '50%', 'display': 'inline-block'}
    ),
    dcc.Dropdown(
        id='region-dropdown',
        options=[
            {'label': 'All Regions', 'value': 'All'},
            {'label': 'Africa', 'value': 'AFRO'},
            {'label': 'Americas', 'value': 'AMRO'},
            {'label': 'Eastern Mediterranean', 'value': 'EMRO'},
            {'label': 'Europe', 'value': 'EURO'},
            {'label': 'South-East Asia', 'value': 'SEARO'},
            {'label': 'Western Pacific', 'value': 'WPRO'}
        ],
        value='All',
        clearable=False,
        style={'width': '50%', 'display': 'inline-block'}
    ),
    dcc.Graph(id='map-graph')
])

# Define the callback to update the graph based on dropdown selections
@app.callback(
    Output('map-graph', 'figure'),
    [Input('map-dropdown', 'value'), Input('region-dropdown', 'value')]
)
def update_map_vaccine(selected_map, selected_region):
    if selected_map == 'booster':
        data = data_booster
        color = 'PERSONS_BOOSTER_ADD_DOSE_PER100'
        title = 'Percentage of total population vaccinated with at least one booster dose<br>World, 31 Dec 2023'
    elif selected_map == 'complete_primary':
        data = data_complete_primary
        color = 'PERSONS_LAST_DOSE_PER100'
        title = 'Percentage of total population vaccinated with a complete primary series<br>World, 31 Dec 2023'
    else:
        data = data_first_dose
        color = 'PERSONS_VACCINATED_1PLUS_DOSE_PER100'
        title = 'Percentage of total population vaccinated with at least one dose of a COVID-19 vaccine<br>World, 31 Dec 2023'
    
    if selected_region != 'All':
        data = data[data['WHO_REGION'] == selected_region]
        region_name = region_names[selected_region]
    else:
        region_name = 'World'

    fig = px.choropleth(
        data,
        locations='ISO3',
        color=color,
        hover_name='COUNTRY',
        hover_data={'ISO3': False, color: False},
        color_continuous_scale='Blues',
        title=title,
        labels={color: ''},
        projection='natural earth'
    )
    
    # Apply continent settings based on selected region
    if region_name != 'World' and region_name in continent_settings:
        fig.update_geos(
            center=continent_settings[region_name]['center'],
            projection_scale=continent_settings[region_name]['zoom']
        )
    
    fig.update_layout(
        width=950,
        height=580,
        coloraxis_colorbar=dict(
            title="% of total population",
            orientation="h",
            yanchor="bottom",
            y=-0.25,
            xanchor="center",
            x=0.4,
            ticks="outside",
            len=0.6
        )
    )
    fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>%{z:.1f}%')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9016)

In [20]:
WHO_vac_booster = WHO_vac[WHO_vac['PERSONS_BOOSTER_ADD_DOSE_PER100'].notna()]
WHO_vac_first_dose = WHO_vac[WHO_vac['PERSONS_VACCINATED_1PLUS_DOSE_PER100'].notna()]
WHO_vac_complete_primary = WHO_vac[WHO_vac['PERSONS_LAST_DOSE_PER100'].notna()]

# Prepare the data for plotting
data_booster = WHO_vac_booster[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_BOOSTER_ADD_DOSE_PER100']]
data_first_dose = WHO_vac_first_dose[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100']]
data_complete_primary = WHO_vac_complete_primary[['ISO3', 'COUNTRY', 'WHO_REGION', 'PERSONS_LAST_DOSE_PER100']]

# Function to create the horizontal bar chart
def create_bar_chart(data, y_column, title, legend_title):
    top_10_data = data.nlargest(10, y_column).sort_values(by=y_column, ascending=True)
    fig = px.bar(
        top_10_data,
        y='COUNTRY',
        x=y_column,
        title=title,
        text=y_column,
        color=y_column,
        orientation='h',
        color_continuous_scale='Blues'
    )
    fig.update_traces(
        texttemplate='%{text:.1f}%',
        textposition='inside',
        insidetextanchor='end',  # Align text to the right inside the bar
        marker_line_width=1.5
    )
    fig.update_layout(
        yaxis_title='Country',
        xaxis_title='% of total population',
        height=500,
        width=800,
        template='plotly_white',
        yaxis=dict(showgrid=False, gridcolor='lightgray', gridwidth=0.1, linecolor='black', linewidth=1),
        xaxis=dict(showgrid=False, gridcolor='lightgray', gridwidth=0.1, linecolor='black', linewidth=1),
        coloraxis_showscale=True,  # Show color scale legend
        coloraxis_colorbar=dict(title=legend_title),
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.2
        )
    )
    return fig

fig_booster = create_bar_chart(data_booster, 'PERSONS_BOOSTER_ADD_DOSE_PER100', 'Top 10 countries by percentage of population vaccinated with at least one booster dose', 'At least one booster')
fig_first_dose = create_bar_chart(data_first_dose, 'PERSONS_VACCINATED_1PLUS_DOSE_PER100', 'Top 10 countries by percentage of population vaccinated with at least one dose of a COVID-19 vaccine', 'At least one dose')
fig_complete_primary = create_bar_chart(data_complete_primary, 'PERSONS_LAST_DOSE_PER100', 'Top 10 countries by percentage of population vaccinated with a complete primary series', 'Complete primary series')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='bar-dropdown',
        options=[
            {'label': 'At least one dose', 'value': 'first_dose'},
            {'label': 'At least one booster', 'value': 'booster'},
            {'label': 'Complete primary series', 'value': 'complete_primary'}
        ],
        value='first_dose',
        clearable=False,
        style={'width': '50%', 'display': 'inline-block'}
    ),
    dcc.Dropdown(
        id='region-dropdown',
        options=[
            {'label': 'All Regions', 'value': 'All'},
            {'label': 'Africa', 'value': 'AFRO'},
            {'label': 'Americas', 'value': 'AMRO'},
            {'label': 'Eastern Mediterranean', 'value': 'EMRO'},
            {'label': 'Europe', 'value': 'EURO'},
            {'label': 'South-East Asia', 'value': 'SEARO'},
            {'label': 'Western Pacific', 'value': 'WPRO'}
        ],
        value='All',
        clearable=False,
        style={'width': '50%', 'display': 'inline-block'}
    ),
    dcc.Graph(id='bar-graph')
])

# Define the callback to update the graph based on dropdown selections
@app.callback(
    Output('bar-graph', 'figure'),
    [Input('bar-dropdown', 'value'), Input('region-dropdown', 'value')]
)
def update_bar(selected_bar, selected_region):
    if selected_bar == 'booster':
        data = data_booster
        y_column = 'PERSONS_BOOSTER_ADD_DOSE_PER100'
        title = 'Top 10 countries by % of population vaccinated with at least one booster dose'
        legend_title = 'At least one booster'
    elif selected_bar == 'complete_primary':
        data = data_complete_primary
        y_column = 'PERSONS_LAST_DOSE_PER100'
        title = 'Top 10 countries by % of population vaccinated with a complete primary series'
        legend_title = 'Complete primary series'
    else:
        data = data_first_dose
        y_column = 'PERSONS_VACCINATED_1PLUS_DOSE_PER100'
        title = 'Top 10 countries by % of population vaccinated with at least one dose of a COVID-19 vaccine'
        legend_title = 'At least one dose'
    
    if selected_region != 'All':
        data = data[data['WHO_REGION'] == selected_region]
    
    fig = create_bar_chart(data, y_column, title, legend_title)
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9083)

<h2 style="text-align: center;">The share of COVID-19 tests that are positive for specific 5 countries, World, Aug 2022</h2>

In [21]:

def plot_covid_positive_rate(countries, start_date='2020-03-05', end_date='2022-08-22'):
    # Convert the date column to datetime
    covid_map['date'] = pd.to_datetime(covid_map['date'])

    # Filter relevant columns and drop missing values
    filtered_data = covid_map[['continent', 'location', 'date', 'positive_rate']].dropna()

    # Filter data to include the specified date range
    filtered_data = filtered_data[(filtered_data['date'] >= start_date) & (filtered_data['date'] <= end_date)]

    # Filter the data to include only the selected countries
    selected_countries_data = filtered_data[filtered_data['location'].isin(countries)]

    # Convert positive_rate to percentage
    selected_countries_data['positive_rate'] = selected_countries_data['positive_rate'] * 100

    # Create an interactive plot with Plotly
    fig = go.Figure()

    # Add traces for each country
    for country in countries:
        country_data = selected_countries_data[selected_countries_data['location'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['date'], 
            y=country_data['positive_rate'], 
            mode='lines', 
            name=country,
            hovertemplate="%{y:.2f}%",
            legendgroup=country
        ))

    # Update layout to set figure size, use template, configure the grid, and set custom ticks
    custom_ticks = pd.date_range(start=start_date, end=end_date, freq='3M')
    custom_tick_labels = [tick.strftime('%b %Y') for tick in custom_ticks]

    fig.update_layout(
        template='plotly_white',
        width=1000,
        height=500,
        xaxis=dict(
            showgrid=False,
            showline=True,
            linecolor='black',
            tickvals=custom_ticks, 
            ticktext=custom_tick_labels,
            range=[pd.Timestamp(start_date), pd.Timestamp(end_date)],
            zeroline=False,
            anchor='y'  # Ensure the x-axis line is anchored to the y-axis
        ),
        yaxis=dict(
            showgrid=True, 
            gridwidth=0.01, 
            gridcolor='LightGrey', 
            griddash='dot', 
            showline=True, 
            linecolor='black',
            tickvals=[i for i in range(0, 101, 10)],  # Tick values from 0 to 100 with steps of 10
            ticktext=[f"{i}%" for i in range(0, 101, 10)],  # Tick text as percentage
            rangemode='tozero'  # Ensure the y-axis starts at zero
        ),
        hovermode='x unified',
        title='The share of COVID-19 tests that are positive, World, Aug 2022',
        xaxis_title='Date',
        yaxis_title='Positive Rate (%)',
        hoverlabel=dict(
            bgcolor="LightGray",
            font_size=12,
            font_family="Rockwell"
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1,
            xanchor="center",
            x=0.5
        )
    )

    # Add country names at the end of each line with slight y offset to avoid overlap
    y_offsets = {country: i * -0.2 for i, country in enumerate(countries)}
    for country in countries:
        country_data = selected_countries_data[selected_countries_data['location'] == country]
        latest_data = country_data[country_data['date'] == country_data['date'].max()]
        fig.add_annotation(
            x=latest_data['date'].values[0],
            y=latest_data['positive_rate'].values[0] + y_offsets[country],
            text=country,
            showarrow=False,
            xanchor='left',
            font=dict(
                size=10,
                color="black"
            ),
            align="left"
        )

    return fig

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H2("COVID-19 Positive Rate Percentage"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in covid_map['location'].unique()],
        value=['Germany', 'France', 'United States', 'Canada', 'India'],  # Set default values
        multi=True
    ),
    dcc.Graph(id='covid-positive-rate')
])

# Define callback to update the graph based on selected countries
@app.callback(
    Output('covid-positive-rate', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_figure(selected_countries):
    return plot_covid_positive_rate(selected_countries)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9030)


<h2 style="text-align: center;">Total COVID-19 Tests per 1,000 People for Specific 5 Countries Over Time</h2>

In [22]:

def visualize_covid_tests(covid_data, countries):
    """
    Visualize total COVID-19 tests per 1,000 people over time for specified countries.
    
    Parameters:
    - covid_data: DataFrame containing the COVID-19 data.
    - countries: List of country names to visualize.
    """
    # Select relevant columns
    covid_data = covid_data[['location', 'total_tests', 'population', 'date']]

    # Drop rows with missing values
    covid_data = covid_data.dropna(subset=['total_tests', 'population'])

    # Convert date to datetime
    covid_data['date'] = pd.to_datetime(covid_data['date'])

    # Calculate tests per 1,000 people
    covid_data['tests_per_1000'] = (covid_data['total_tests'] / covid_data['population']) * 1000

    # Filter the data to include only the specified countries
    filtered_data = covid_data[covid_data['location'].isin(countries)]

    # Resample data to a monthly frequency
    filtered_data.set_index('date', inplace=True)
    filtered_data = filtered_data.groupby('location').resample('M').mean().reset_index()

    # Create an interactive plot with Plotly
    fig = go.Figure()

    # Add traces for each country
    for country in countries:
        country_data = filtered_data[filtered_data['location'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['date'], 
            y=country_data['tests_per_1000'], 
            mode='lines', 
            name=country,
            hovertemplate="%{y:.2f}",
            legendgroup=country
        ))

    # Update layout to set figure size, use template, configure the grid, and set custom ticks
    start_date = filtered_data['date'].min()
    end_date = filtered_data['date'].max()
    custom_ticks = pd.date_range(start=start_date, end=end_date, freq='3M')
    custom_tick_labels = [tick.strftime('%b %Y') for tick in custom_ticks]

    fig.update_layout(
        template='plotly_white',
        width=900,
        height=500,
        xaxis=dict(
            showgrid=False,
            showline=True,
            linecolor='black',
            tickvals=custom_ticks, 
            ticktext=custom_tick_labels,
            range=[pd.Timestamp(start_date), pd.Timestamp(end_date)],
            zeroline=False,
            anchor='y'  # Ensure the x-axis line is anchored to the y-axis
        ),
        yaxis=dict(
            showgrid=True, 
            gridwidth=0.01, 
            gridcolor='LightGrey', 
            griddash='dot', 
            showline=True, 
            linecolor='black',
            rangemode='tozero'  # Ensure the y-axis starts at zero
        ),
        hovermode='x unified',
        title='Total COVID-19 Tests per 1,000 People Over Time',
        xaxis_title='Date',
        yaxis_title='Total Tests per 1,000 People',
        hoverlabel=dict(
            bgcolor="LightGray",
            font_size=12,
            font_family="Rockwell"
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1,
            xanchor="center",
            x=0.5
        )
    )

    # Add country names at the end of each line with slight y offset to avoid overlap
    y_offsets = {country: i * -0.2 for i, country in enumerate(countries)}
    for country in countries:
        country_data = filtered_data[filtered_data['location'] == country]
        latest_data = country_data[country_data['date'] == country_data['date'].max()]
        fig.add_annotation(
            x=latest_data['date'].values[0],
            y=latest_data['tests_per_1000'].values[0] + y_offsets[country],
            text=country,
            showarrow=False,
            xanchor='left',
            font=dict(
                size=10,
                color="black"
            ),
            align="left"
        )

    return fig

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in covid_map['location'].unique()],
        value=['Germany', 'France', 'United States', 'Italy', 'India'],  # Set default values
        multi=True
    ),
    dcc.Graph(id='covid-tests')
])

# Define callback to update the graph based on selected countries
@app.callback(
    Output('covid-tests', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_figure(selected_countries):
    return visualize_covid_tests(covid_map, selected_countries)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True,port=9031)


<h2 style="text-align: center;">Highest Strictness 5 Countries, COVID-19: Stringency Index, Dec 31, 2022.</h2>

In [23]:
def plot_stringency_index(filtered_data, countries):
    # Create the interactive plot
    fig = go.Figure()
    
    for country in countries:
        country_data = filtered_data[filtered_data['location'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['date'],
            y=country_data['stringency_index'],
            mode='lines',
            name=country,
        ))
    
    # Custom tick values and labels
    custom_ticks = [
        "2020-01-31", "2020-08-08", "2021-02-24", 
        "2021-09-12", "2022-03-31", "2022-12-31"
    ]
    custom_tick_labels = [
        "Jan 1, 2020", "Aug 8, 2020", "Feb 24, 2021", 
        "Sep 12, 2021", "Mar 31, 2022", "Dec 31, 2022"
    ]
    
    # Update layout to set figure size, use template, configure the grid, and set custom ticks
    fig.update_layout(
        template='plotly_white',
        width=1000,
        height=500,
        xaxis=dict(
            showgrid=False, gridwidth=0.1, gridcolor='LightGrey', 
            showline=True, linecolor='black',
            tickvals=custom_ticks, ticktext=custom_tick_labels
        ),
        yaxis=dict(
            showgrid=True, gridwidth=0.01, gridcolor='LightGrey', griddash='dot', 
            showline=True, linecolor='black', rangemode='tozero'
        ),
        hovermode='x unified',
        title='COVID-19 Stringency Index till Dec 31, 2022',
        yaxis_title='Stringency Index (0-100; 100 = strictest)',
        hoverlabel=dict(
            bgcolor="LightGrey",
            font_size=12,
            font_family="Rockwell"
        )
    )
    
    # Add country names at the end of each line with slight y offset to avoid overlap
    y_offsets = {country: i * -.1 for i, country in enumerate(countries)}
    for country in countries:
        country_data = filtered_data[filtered_data['location'] == country]
        fig.add_annotation(
            x=country_data['date'].max(),
            y=country_data['stringency_index'].iloc[-1] + y_offsets[country],
            text=country,
            showarrow=False,
            xanchor='left'
        )
    
    return fig

app = dash.Dash(__name__)



# Convert the date column to datetime
covid_map['date'] = pd.to_datetime(covid_map['date'])

# Filter data to include only dates till December 31, 2022
covid_map = covid_map[covid_map['date'] <= '2022-12-31']

# Get unique list of countries for the dropdown options
countries_options = [{'label': country, 'value': country} for country in covid_map['location'].unique()]

# Layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='country-select',
        options=countries_options,
        value=['India', 'Germany','Italy'],  # Default selected countries
        multi=True  # Allow multiple selections
    ),
    dcc.Graph(id='stringency-index-plot')
])

# Callback to update the plot based on the selected countries
@app.callback(
    Output('stringency-index-plot', 'figure'),
    [Input('country-select', 'value')]
)
def update_plot(selected_countries):
    if not selected_countries:
        selected_countries = []
    filtered_data = covid_map[covid_map['location'].isin(selected_countries)]
    return plot_stringency_index(filtered_data, selected_countries)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9018)

<h1 style="text-align: center;">COVID-19 hospitalizations</h1>
<h2 style="text-align: center;">How many people are in intensive care (ICU) due to COVID-19 at a given time?</h2>

<h2 style="text-align: center;">How many people are in hospital due to COVID-19 at a given time?</h2>


In [24]:
# Filter the data for the relevant indicators
filtered_data_per_million = covid_data[covid_data['indicator'] == 'Daily ICU occupancy per million']
filtered_data_total = covid_data[covid_data['indicator'] == 'Daily ICU occupancy']
filtered_data_hospital_per_million = covid_data[covid_data['indicator'] == 'Daily hospital occupancy per million']
filtered_data_hospital_total = covid_data[covid_data['indicator'] == 'Daily hospital occupancy']
filtered_data_weekly_hospital_admissions = covid_data[covid_data['indicator'] == 'Weekly new hospital admissions']
filtered_data_weekly_hospital_admissions_per_million = covid_data[covid_data['indicator'] == 'Weekly new hospital admissions per million']

# Convert date column to datetime
filtered_data_per_million['date'] = pd.to_datetime(filtered_data_per_million['date'])
filtered_data_total['date'] = pd.to_datetime(filtered_data_total['date'])
filtered_data_hospital_per_million['date'] = pd.to_datetime(filtered_data_hospital_per_million['date'])
filtered_data_hospital_total['date'] = pd.to_datetime(filtered_data_hospital_total['date'])
filtered_data_weekly_hospital_admissions['date'] = pd.to_datetime(filtered_data_weekly_hospital_admissions['date'])
filtered_data_weekly_hospital_admissions_per_million['date'] = pd.to_datetime(filtered_data_weekly_hospital_admissions_per_million['date'])

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='indicator-dropdown',
        options=[
            {'label': 'ICU Patients per Million', 'value': 'per_million_icu'},
            {'label': 'Daily ICU Occupancy', 'value': 'total_icu'},
            {'label': 'Daily Hospital Occupancy per Million', 'value': 'per_million_hospital'},
            {'label': 'Daily Hospital Occupancy', 'value': 'total_hospital'},
            {'label': 'Weekly New Hospital Admissions', 'value': 'weekly_hospital_admissions'},
            {'label': 'Weekly New Hospital Admissions per Million', 'value': 'weekly_hospital_admissions_per_million'}
        ],
        value='per_million_icu',
        clearable=False
    ),
    dcc.Dropdown(
        id='countries-dropdown',
        options=[{'label': country, 'value': country} for country in covid_data['entity'].unique()],
        value=["Germany", "France", "Italy", "Sweden", "Netherlands"],
        multi=True
    ),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=covid_data['date'].min(),
        end_date=covid_data['date'].max(),
        display_format='YYYY-MM-DD'
    ),
    html.Div([
        dcc.Checklist(
            id='log-scale-checklist',
            options=[{'label': 'Logarithmic Scale', 'value': 'log'}],
            value=[]
        ),
        dcc.Checklist(
            id='moving-average-checklist',
            options=[{'label': '7-Day Moving Average', 'value': 'moving_average'}],
            value=[]
        )
    ], style={'display': 'flex', 'gap': '20px'}),
    dcc.Graph(id='icu-plot')
])

# Callback to update the plot based on the selected indicator, countries, date range, and options
@app.callback(
    Output('icu-plot', 'figure'),
    [Input('indicator-dropdown', 'value'),
     Input('countries-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('log-scale-checklist', 'value'),
     Input('moving-average-checklist', 'value')]
)
def update_figure(selected_indicator, selected_countries, start_date, end_date, log_scale, moving_average):
    if selected_indicator == 'per_million_icu':
        filtered_data = filtered_data_per_million
        yaxis_title = 'ICU Patients per Million'
    elif selected_indicator == 'total_icu':
        filtered_data = filtered_data_total
        yaxis_title = 'Daily ICU Occupancy'
    elif selected_indicator == 'per_million_hospital':
        filtered_data = filtered_data_hospital_per_million
        yaxis_title = 'Daily Hospital Occupancy per Million'
    elif selected_indicator == 'total_hospital':
        filtered_data = filtered_data_hospital_total
        yaxis_title = 'Daily Hospital Occupancy'
    elif selected_indicator == 'weekly_hospital_admissions':
        filtered_data = filtered_data_weekly_hospital_admissions
        yaxis_title = 'Weekly New Hospital Admissions'
    elif selected_indicator == 'weekly_hospital_admissions_per_million':
        filtered_data = filtered_data_weekly_hospital_admissions_per_million
        yaxis_title = 'Weekly New Hospital Admissions per Million'
    else:
        filtered_data = pd.DataFrame()
        yaxis_title = 'Unknown Indicator'
    
    filtered_data_countries = filtered_data[(filtered_data['entity'].isin(selected_countries)) & 
                                            (filtered_data['date'] >= start_date) & 
                                            (filtered_data['date'] <= end_date)]
    
    pivot_data_countries = filtered_data_countries.pivot(index='date', columns='entity', values='value')

    fig = go.Figure()
    for country in pivot_data_countries.columns:
        data = pivot_data_countries[country]
        if 'moving_average' in moving_average:
            data = data.rolling(window=7).mean()
        fig.add_trace(go.Scatter(x=pivot_data_countries.index, y=data, mode='lines', name=country))

    yaxis_type = 'log' if 'log' in log_scale else 'linear'
    yaxis_tickvals = [50, 100, 200, 500, 1000, 10000, 20000, 40000] if 'log' in log_scale else None

    fig.update_layout(
        title='Number of COVID-19 Patients',
        xaxis_title='Date',
        yaxis_title=yaxis_title,
        legend_title='Country',
        hovermode='x unified',
        template='plotly_white',
        yaxis=dict(
            showgrid=True,
            gridwidth=0.001,
            gridcolor='lightgray',
            griddash='dot',
            showline=True,
            linecolor='black',
            rangemode='tozero',
            type=yaxis_type,
            tickvals=yaxis_tickvals
        ),
        xaxis=dict(
            linecolor='black',
            showgrid=False
        )
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9019)


<h1 style="text-align: center;">Conclusion</h1>


In conclusion, our analysis reveals several key insights, including global distribution and impact, temporal trends, testing and vaccination efforts, healthcare system burdens, comparative analysis, and policy stringency. Utilizing data from reputable sources such as WHO and Johns Hopkins University, we highlighted the extensive global impact, with the United States, China, and India being the most affected. Significant regional disparities are evident, with Asia and Europe experiencing the highest numbers of confirmed cases and recoveries, while North America and Europe recorded the highest death rates.
Our analysis of weekly trends in new cases and deaths revealed significant fluctuations, with major spikes around mid-2022 and early 2023, emphasizing the persistent and unpredictable nature of the pandemic. Death rates showed notable peaks in early 2021 and early 2022, with weekly tolls exceeding 100,000.
Vaccination efforts varied widely, with North America and Europe achieving high vaccination rates, contrasting with lower rates in Africa and Asia. Testing efforts also showed significant disparities. Healthcare systems were heavily burdened, particularly in France and Germany, where ICU occupancy peaked significantly. Comparative analyses of countries like Germany and Italy highlighted varied responses and outcomes.
Overall, the detailed insights provided here offer valuable guidance for policymakers and healthcare professionals, helping them make informed decisions. 


<h1 style="text-align: center;">References</h1>

[1] https://data.who.int/dashboards/covid19/data?n=c&m49=001

[2] https://www.worldometers.info/coronavirus/

[3] https://github.com/owid/covid-19-data

[4] https://covid.ourworldindata.org/data/owid-covid-data.csv

[5] https://github.com/CSSEGISandData/COVID-19/tree/master
